# 3: Additional good outcomes

## Plain English summary
This notebook builds on the scenarios that we considered in a previous notebook. Now that we know the effect of changing scenario on the proportions of patients who meet certain criteria, we can feed that data into a pathway simulator and an outcome model.

The pathway simulator generates a length of time for each patient and for each step in the pathway. The times are selected using the expected distributions of times at each step that we measured previously.

The outcome model takes the final times from onset to treatment for each patient and calculates their outcome. The outcomes are given as mRS (modified Rankin Scale) scores, where 0 is completely independent and 6 is dead. 

We run the pathway and outcome models to generate outcomes for 100 years' worth of patients. By averaging the results over the 100 years, we can find the effect of scenario on the number of additional good outcomes. Here we define additional good outcomes as the extra number compared with the case where nobody receives any treatment for their stroke.

## Notebook setup:

In [1]:
import pandas as pd
import numpy as np
import pickle
from dataclasses import dataclass
from math import sqrt

import matplotlib.pyplot as plt

import stroke_utilities.scenario
from classes.pathway import SSNAP_Pathway

import warnings
warnings.filterwarnings("ignore")

## Set up paths and filenames

In [2]:
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    output_folder = './stroke_utilities/output/'

paths = Paths()

## Import existing utilities

This hospital performance data for various scenarios was calculated in a previous notebook.

In [3]:
filename = paths.output_folder + 'all_performance_scenarios.csv'
df_performance_scenarios = pd.read_csv(filename, index_col=0)

In [4]:
df_performance_scenarios.T

,1 / lvo / base,1 / nlvo / base,1 / other / base,1 / mixed / base,1 / lvo / benchmark,1 / nlvo / benchmark,1 / other / benchmark,1 / mixed / benchmark,1 / lvo / onset,1 / nlvo / onset,...,119 / other / speed + benchmark,119 / mixed / speed + benchmark,119 / lvo / speed + onset,119 / nlvo / speed + onset,119 / other / speed + onset,119 / mixed / speed + onset,119 / lvo / speed + onset + benchmark,119 / nlvo / speed + onset + benchmark,119 / other / speed + onset + benchmark,119 / mixed / speed + onset + benchmark
stroke_type,lvo,nlvo,other,mixed,lvo,nlvo,other,mixed,lvo,nlvo,...,other,mixed,lvo,nlvo,other,mixed,lvo,nlvo,other,mixed
admissions,154.833333,676.666667,101.166667,932.666667,154.833333,676.666667,101.166667,932.666667,154.833333,676.666667,...,65.833333,563.166667,93.833333,403.5,65.833333,563.166667,93.833333,403.5,65.833333,563.166667
proportion_of_all_with_ivt,0.341227,0.09064,0.0,0.122409,0.345533,0.103198,0.0,0.133667,0.300324,0.088913,...,0.0,0.116378,0.230206,0.08918,0.0,0.09992,0.289233,0.110257,0.0,0.127406
proportion_of_all_with_mt,0.176534,0.014778,0.0,0.040029,0.152852,0.009606,0.0,0.032344,0.152852,0.010939,...,0.0,0.006422,0.029337,0.003216,0.0,0.00703,0.029337,0.003216,0.0,0.00703
proportion_of_mt_with_ivt,0.77439,0.466667,NaN,0.691964,0.77439,0.466667,NaN,0.691964,0.77439,0.466667,...,NaN,0.485714,0.565217,0.333333,NaN,0.485714,0.565217,0.333333,NaN,0.485714
proportion1_of_all_with_onset_known_ivt,0.481163,0.337931,0.37397,0.365618,0.481163,0.337931,0.37397,0.365618,0.481163,0.384824,...,0.379747,0.355135,0.43389,0.384824,0.398444,0.388787,0.43389,0.384824,0.398444,0.388787
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,0.897092,0.653061,0.762115,0.718475,0.897092,0.653061,0.762115,0.718475,0.897092,0.653061,...,0.786667,0.726667,0.88785,0.674641,0.786667,0.726667,0.88785,0.674641,0.786667,0.726667
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,1.0,0.944196,0.942197,0.959184,1.0,0.944196,0.942197,0.959184,1.0,0.944196,...,0.95,0.95,0.95,0.95,0.95,0.95,0.95,0.95,0.95,0.95
proportion4_of_mask3_with_onset_to_scan_on_time_ivt,0.972569,0.885343,0.91411,0.913475,0.972569,0.885343,0.91411,0.913475,0.972569,0.885343,...,0.911504,0.892771,0.962366,0.864407,0.911504,0.892771,0.962366,0.864407,0.911504,0.892771
proportion5_of_mask4_with_enough_time_to_treat_ivt,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Run models

Find the complete list of stroke teams and scenarios:

In [5]:
stroke_teams = sorted(list(set(df_performance_scenarios['stroke_team'])))

In [6]:
scenario_list = sorted(list(set(df_performance_scenarios['scenario'])))

The following cell looks at each stroke team in turn. It picks out the patients with each stroke type. 

Then separate patient pathways are run for the separate stroke types. The results from the pathways are combined into a single data set.

Some of the pathway results are split off and used for the outcome model. The required results for each patient are their times from onset to treatment, which treatment they received, and which stroke type they have. 

Then the outcome model is run to find out the expected outcome of patients with those features who were treated at those times. The "discrete" outcome model is used, and so the patient must have a pre-stroke mRS score. Rather than sampling this from the original data where there might not be many patients for each stroke type, we create a more representative selection by sampling uniformly from a pre-stroke mRS distribution.

In [7]:
np.random.seed(42)

# How many trials to run:
n_trials = 100

results_df, outcome_results_columns, trial_columns = stroke_utilities.scenario.set_up_results_dataframe()

for stroke_team in stroke_teams:
    for scenario in scenario_list:
        # Get data for one hospital.
        # Squeeze to convert DataFrame to Series.
        lvo_data = df_performance_scenarios[(
            (df_performance_scenarios['stroke_team'] == stroke_team) & 
            (df_performance_scenarios['stroke_type'] == 'lvo') &
            (df_performance_scenarios['scenario'] == scenario)
            )].copy().squeeze()
        nlvo_data = df_performance_scenarios[(
            (df_performance_scenarios['stroke_team'] == stroke_team) & 
            (df_performance_scenarios['stroke_type'] == 'nlvo') &
            (df_performance_scenarios['scenario'] == scenario)
            )].copy().squeeze()
        other_data = df_performance_scenarios[(
            (df_performance_scenarios['stroke_team'] == stroke_team) & 
            (df_performance_scenarios['stroke_type'] == 'other') &
            (df_performance_scenarios['scenario'] == scenario)
            )].copy().squeeze()
    
        # Set up trial results dataframe
        trial_df = pd.DataFrame(columns=trial_columns)
        # Set up the pathways with this data...
        pathway_object_dict = stroke_utilities.scenario.set_up_pathway_objects(stroke_team, lvo_data, nlvo_data, other_data)
        for trial in range(n_trials):
            # ... run the pathways...
            combo_trial_dict = stroke_utilities.scenario.run_trial_of_pathways(pathway_object_dict)
            # ... overwrite the results so that nobody has thrombectomy...
            combo_trial_dict['mt_chosen_bool'] = np.array([0] * len(combo_trial_dict['mt_chosen_bool'])) == 1
            # ... and run the clinical outcome model.
            results_by_stroke_type, patient_array_outcomes = (
                stroke_utilities.scenario.run_discrete_outcome_model(combo_trial_dict))

            number_of_patients = len(combo_trial_dict['stroke_type_code'])
            # Patients' mRS if not treated..
            mrs_not_treated = patient_array_outcomes['each_patient_mrs_not_treated']
            # Patients' mRS in this trial...
            mrs_post_stroke = patient_array_outcomes['each_patient_mrs_post_stroke']
            # How many patients were good outcomes? mRS 0 or 1.
            n_good_baseline = len(np.where(mrs_not_treated <= 1)[0])
            n_good_post_stroke = len(np.where(mrs_post_stroke <= 1)[0])
            # Additional good outcomes:
            n_good_additional = n_good_post_stroke - n_good_baseline
            # Convert to outcomes per 1000 patients:
            n_baseline_good_per_1000 = n_good_baseline * (1000.0 / number_of_patients)
            n_additional_good_per_1000 = n_good_additional * (1000.0 / number_of_patients)
    
            result = stroke_utilities.scenario.gather_results_from_trial(
                trial_columns, combo_trial_dict, results_by_stroke_type,
                n_baseline_good_per_1000, n_additional_good_per_1000
                )
            trial_df.loc[trial] = result
    
        summary_trial_results = stroke_utilities.scenario.gather_summary_results_across_all_trials(outcome_results_columns, trial_df)
        summary_trial_results += [f'{stroke_team}']
        summary_trial_results += [scenario.replace(' + ', '_')]
        
        # add scenario results to results dataframe
        results_df.loc[f'{stroke_team} / {scenario}'] = summary_trial_results

## Results

In [8]:
results_df.T

,1 / base,1 / benchmark,1 / onset,1 / onset + benchmark,1 / speed,1 / speed + benchmark,1 / speed + onset,1 / speed + onset + benchmark,2 / base,2 / benchmark,...,118 / speed + onset,118 / speed + onset + benchmark,119 / base,119 / benchmark,119 / onset,119 / onset + benchmark,119 / speed,119 / speed + benchmark,119 / speed + onset,119 / speed + onset + benchmark
Percent_Thrombolysis_(median),11.278195,13.748657,11.707841,14.607948,11.493018,14.393126,12.406015,15.46724,12.006861,11.835334,...,12.421053,13.368421,9.536542,11.675579,10.338681,13.458111,9.893048,11.942959,11.051693,14.26025
Percent_Thrombolysis_(low_5%),9.661654,12.244898,10.306122,13.426423,10.0913,12.776584,10.741139,13.641246,9.768439,10.454545,...,9.473684,11.157895,7.834225,9.616756,8.377897,11.568627,8.368984,10.516934,9.26025,11.764706
Percent_Thrombolysis_(high_95%),12.674544,15.682062,13.002148,16.331901,12.889366,16.444683,14.070892,17.406015,14.06518,14.073756,...,15.578947,16.431579,11.595365,13.912656,12.673797,15.329768,12.317291,14.795009,13.368984,16.586453
Percent_Thrombolysis_(mean),11.236305,13.900107,11.716434,14.75725,11.501611,14.484425,12.389903,15.537057,11.969125,12.046312,...,12.593684,13.597895,9.557932,11.643494,10.500891,13.409982,10.067736,12.27451,11.087344,14.139037
Percent_Thrombolysis_(stdev),0.950214,1.032863,0.893815,0.993972,0.846953,1.136876,1.110115,1.126062,1.307349,1.212841,...,1.666676,1.656361,1.219486,1.270049,1.30032,1.171176,1.308534,1.285906,1.372115,1.47294
Percent_Thrombolysis_(95ci),0.186239,0.202437,0.175185,0.194815,0.166,0.222824,0.217578,0.220704,0.256236,0.237712,...,0.326663,0.324641,0.239015,0.248925,0.254858,0.229546,0.256468,0.252033,0.26893,0.288691
Baseline_good_outcomes_per_1000_patients_(median),355.531686,355.531686,354.457573,352.309345,353.383459,351.235231,355.531686,360.902256,355.060034,353.344768,...,328.421053,330.526316,349.376114,354.723708,350.26738,351.158645,351.158645,352.941176,352.049911,350.26738
Baseline_good_outcomes_per_1000_patients_(low_5%),329.752954,332.92159,329.752954,331.901182,326.423201,328.67884,337.164339,336.143931,327.530017,324.099485,...,296.842105,298.842105,320.855615,331.461676,320.855615,322.54902,318.805704,320.855615,320.766488,324.242424
Baseline_good_outcomes_per_1000_patients_(high_95%),386.734694,380.558539,377.013963,374.919441,380.290011,377.228786,378.195489,382.706767,382.590051,381.132075,...,372.631579,360.105263,383.244207,390.463458,377.98574,385.115865,381.461676,376.292335,379.768271,376.203209
Baseline_good_outcomes_per_1000_patients_(mean),356.928034,355.145005,354.704619,352.771214,353.941998,353.061224,355.864662,360.665951,351.93825,352.538593,...,330.463158,330.147368,350.766488,357.112299,349.946524,353.048128,350.534759,352.049911,351.158645,350.017825


Save the results to file:

In [9]:
df = results_df

# Round the values to fewer decimal places:
for column in df.columns:
    if column not in ['stroke_team', 'stroke_team_id', 'scenario']:
        df[column] = df[column].astype(float).round(6)
        
df.to_csv(
    f'{paths.output_folder}/scenario_results.csv',
    index=False
    )